# Automated ML

TODO: Import Dependencies. In the cell below, import all the dependencies that you will need to complete the project.

In [1]:
pip install --upgrade azureml-core

     |████████████████████████████████| 3.3 MB 3.7 MB/s eta 0:00:01
     |████████████████████████████████| 121 kB 60.0 MB/s eta 0:00:01
ERROR: ray 2.0.0 has requirement click<=8.0.4,>=7.0, but you'll have click 8.1.3 which is incompatible.
ERROR: ray 2.0.0 has requirement grpcio<=1.43.0,>=1.28.1; python_version < "3.10", but you'll have grpcio 1.54.2 which is incompatible.
ERROR: jupyterlab-server 2.23.0 has requirement jinja2>=3.0.3, but you'll have jinja2 2.11.2 which is incompatible.
ERROR: google-auth 2.18.1 has requirement urllib3<2.0, but you'll have urllib3 2.2.2 which is incompatible.
ERROR: datasets 2.3.2 has requirement dill<0.3.6, but you'll have dill 0.3.6 which is incompatible.
ERROR: databricks-cli 0.17.7 has requirement urllib3<2.0.0,>=1.26.7, but you'll have urllib3 2.2.2 which is incompatible.
ERROR: dask-sql 2023.6.0 has requirement pandas>=1.4.0, but you'll have pandas 1.1.5 which is incompatible.
ERROR: botocore 1.23.19 has requirement urllib3<1.27,>=1.25.4, but yo

In [2]:
from azureml.core import Workspace, Experiment
from azureml.core.compute import ComputeTarget, AmlCompute
from azureml.core.compute_target import ComputeTargetException
from azureml.widgets import RunDetails
from azureml.train.sklearn import SKLearn
from azureml.train.hyperdrive.run import PrimaryMetricGoal
from azureml.train.hyperdrive.policy import BanditPolicy
from azureml.train.hyperdrive.sampling import RandomParameterSampling
from azureml.train.hyperdrive.runconfig import HyperDriveConfig
from azureml.train.hyperdrive.parameter_expressions import choice, loguniform
from azureml.core import Environment, ScriptRunConfig
import os
import pandas as pd
from azureml.train.automl import AutoMLConfig
from azureml.core.dataset import Dataset
import logging
import csv
from matplotlib import pyplot as plt
import numpy as np
import pandas as pd
from sklearn import datasets
import pkg_resources
import azureml.core
from azureml.core.experiment import Experiment
from azureml.core.workspace import Workspace
from azureml.train.automl import AutoMLConfig
from azureml.core.dataset import Dataset
from azureml.pipeline.steps import AutoMLStep
import joblib
from azureml.pipeline.core import PipelineData, TrainingOutput
from azureml.core import Model

## Dataset

### Overview
TODO: In this markdown cell, give an overview of the dataset you are using. Also mention the task you will be performing.


TODO: Get data. In the cell below, write code to access the data you will be using in this project. Remember that the dataset needs to be external.

In [3]:
ws = Workspace.get(name = 'quick-starts-ws-260824',
                   subscription_id = '3e42d11f-d64d-4173-af9b-12ecaa1030b3',
                   resource_group = 'aml-quickstarts-260824')

# choose a name for experiment
experiment_name = 'automl-heart'

experiment=Experiment(ws, experiment_name)

print('Workspace name: ' + ws.name, 
      'Azure region: ' + ws.location, 
      'Subscription id: ' + ws.subscription_id, 
      'Resource group: ' + ws.resource_group, sep = '\n')

run = experiment.start_logging()

Workspace name: quick-starts-ws-260824
Azure region: southcentralus
Subscription id: 3e42d11f-d64d-4173-af9b-12ecaa1030b3
Resource group: aml-quickstarts-260824


## AutoML Configuration

TODO: Explain why you chose the automl settings and cofiguration you used below.

In [4]:
cluster_name = "notebook260824"
project_folder = './capstone'

found = False
description_text = "Heart Data for Udacity Capstone"

dataset = Dataset.get_by_name(ws, name='heart-data')

try:
    aml_compute = ComputeTarget(workspace=ws, name = cluster_name)
    print("Found existing cluster, use it")
except ComputeTargetException:
    compute_config = AmlCompute.provisioning_configuration(vm_size="STANDARD_D2_V2", max_nodes= 4)
    aml_compute = ComputeTarget.create(ws, cluster_name, compute_config)

# TODO: Put your automl settings here
automl_settings = {
    "experiment_timeout_minutes": 20,
    "max_concurrent_iterations": 4,
    "primary_metric" : 'accuracy',
    "n_cross_validations": 5
}

# TODO: Put your automl config here
automl_config = AutoMLConfig(compute_target=cluster_name,
                             task = "classification",
                             training_data=dataset,
                             #time_column_name="date", 
                             label_column_name="DEATH_EVENT",  
                             #path = project_folder,
                             #enable_early_stopping= True,
                             featurization= 'auto',
                             debug_log = "automl_errors.log",
                             **automl_settings
                            )

Found existing cluster, use it


In [5]:
# TODO: Submit your experiment
remote_run = experiment.submit(automl_config)

Submitting remote run.


Experiment,Id,Type,Status,Details Page,Docs Page
automl-heart,AutoML_b39ec310-aa0a-48c0-929f-41caee24e096,automl,NotStarted,Link to Azure Machine Learning studio,Link to Documentation


## Run Details

OPTIONAL: Write about the different models trained and their performance. Why do you think some models did better than others?

TODO: In the cell below, use the `RunDetails` widget to show the different experiments.

In [23]:
# remote_run = experiment.submit(automl_config, show_output = False)

# RunDetails(remote_run).show()

# automl_run = experiment.start_logging()

# automl_run.log(name="message", value = "Hello from run!")

# print(automl_run.get_status())

RunDetails(remote_run).show()

_AutoMLWidget(widget_settings={'childWidgetDisplay': 'popup', 'send_telemetry': False, 'log_level': 'INFO', 's…

## Best Model

TODO: In the cell below, get the best model from the automl experiments and display all the properties of the model.



In [24]:
best_run_metrics_automl = remote_run.get_metrics()
print(best_run_metrics_automl["accuracy"])

0.8897175141242938


In [9]:
#TODO: Save the best model
model = remote_run.register_model(model_name = "automl-heart-sdk", metric="accuracy")

## Model Deployment

Remember you have to deploy only one of the two models you trained but you still need to register both the models. Perform the steps in the rest of this notebook only if you wish to deploy this model.

TODO: In the cell below, register the model, create an inference config and deploy the model as a web service.

In [15]:
from azureml.core.model import Model, InferenceConfig
from azureml.core import Environment
from azureml.core.webservice import AciWebservice

#print(remote_run.get_environment())

#env = Environment.from_conda_specification(name='sklearn-1.0:37', file_path='conda_dependencies.yml')
#env = Environment.get(workspace = ws, name= "heart-automl-env")

# get the best run
best_run, best_model = remote_run.get_output()
# get the environment from the best run
env = best_run.get_environment()

#env = Environment.get(workspace=ws, name="AzureML-AutoML")

best_run.download_file('outputs/scoring_file_v_1_0_0.py', 'score_new.py')

Package:azureml-automl-runtime, training version:1.52.0.post1, current version:1.51.0.post1
Package:azureml-core, training version:1.52.0, current version:1.51.0
Package:azureml-dataprep, training version:4.11.4, current version:4.10.8
Package:azureml-dataprep-rslex, training version:2.18.4, current version:2.17.12
Package:azureml-dataset-runtime, training version:1.52.0, current version:1.51.0
Package:azureml-defaults, training version:1.52.0, current version:1.51.0
Package:azureml-interpret, training version:1.52.0, current version:1.51.0
Package:azureml-mlflow, training version:1.52.0, current version:1.51.0
Package:azureml-pipeline-core, training version:1.52.0, current version:1.51.0
Package:azureml-responsibleai, training version:1.52.0, current version:1.51.0
Package:azureml-telemetry, training version:1.52.0, current version:1.51.0
Package:azureml-train-automl-client, training version:1.52.0, current version:1.51.0.post1
Package:azureml-train-automl-runtime, training version:1.

In [16]:
inference_config = InferenceConfig(entry_script='score_new.py', environment=env)

#inf_config = InferenceConfig(entry_script='score.py', environment=env, conda_file="conda_dependencies.yml")
#inf_config = InferenceConfig(entry_script='score.py', environment=env)

deployment_config = AciWebservice.deploy_configuration(cpu_cores = 1, memory_gb = 4, enable_app_insights=True, auth_enabled=True)
#deployment_config = AciWebservice.deploy_configuration(cpu_cores = 1, memory_gb = 4)

service = Model.deploy(workspace = ws, 
                    name = "heart-automl-deploy",
                    models = [model],
                    inference_config= inference_config,
                    deployment_config=deployment_config,
                    overwrite = True)

service.wait_for_deployment(show_output=True)

# ensure app insights is enabled
if service.enable_app_insights==False:
    service.update(enable_app_insights=True)
# print the sevice logs
print(service.get_logs())

Tips: You can try get_logs(): https://aka.ms/debugimage#dockerlog or local deployment: https://aka.ms/debugimage#debug-locally to debug if deployment takes longer than 10 minutes.
Running
2024-06-18 15:36:39+00:00 Creating Container Registry if not exists.
2024-06-18 15:36:39+00:00 Registering the environment.
2024-06-18 15:36:40+00:00 Use the existing image.
2024-06-18 15:36:40+00:00 Generating deployment configuration.
2024-06-18 15:36:41+00:00 Submitting deployment to compute.
2024-06-18 15:36:43+00:00 Checking the status of deployment heart-automl-deploy..
2024-06-18 15:42:48+00:00 Checking the status of inference endpoint heart-automl-deploy.
Succeeded
ACI service creation operation finished, operation "Succeeded"
2024-06-18T15:42:15,257116293+00:00 - rsyslog/run 
2024-06-18T15:42:15,259884673+00:00 - gunicorn/run 
2024-06-18T15:42:15,262051055+00:00 | gunicorn/run | 
2024-06-18T15:42:15,263618715+00:00 | gunicorn/run | ###############################################
2024-06-18T15

TODO: In the cell below, send a request to the web service you deployed to test it.

In [17]:
scoring_uri = service.scoring_uri
print(scoring_uri)

key = service.get_keys()[0]
print(key)

http://3a51d690-8529-43c6-bdfd-4ae7ec2d19c1.southcentralus.azurecontainer.io/score
bidYWKcZvZq3r3piukoLRnsXW2h0sILk


In [20]:
%run endpoint.py

{"result": [1, 1]}


TODO: In the cell below, print the logs of the web service and delete the service

In [21]:
# getting and printing all logs
endpoint_logs = service.get_logs()
print(endpoint_logs)

2024-06-18T15:42:15,257116293+00:00 - rsyslog/run 
2024-06-18T15:42:15,259884673+00:00 - gunicorn/run 
2024-06-18T15:42:15,262051055+00:00 | gunicorn/run | 
2024-06-18T15:42:15,263618715+00:00 | gunicorn/run | ###############################################
2024-06-18T15:42:15,266677124+00:00 | gunicorn/run | AzureML Container Runtime Information
2024-06-18T15:42:15,269510811+00:00 | gunicorn/run | ###############################################
2024-06-18T15:42:15,270277351+00:00 - nginx/run 
2024-06-18T15:42:15,271488240+00:00 | gunicorn/run | 
2024-06-18T15:42:15,276889841+00:00 | gunicorn/run | 
2024-06-18T15:42:15,286533905+00:00 | gunicorn/run | AzureML image information: openmpi4.1.0-ubuntu20.04, Materializaton Build:20230628.v2
2024-06-18T15:42:15,290553346+00:00 | gunicorn/run | 
2024-06-18T15:42:15,292591305+00:00 | gunicorn/run | 
2024-06-18T15:42:15,293463947+00:00 | gunicorn/run | PATH environment variable: /azureml-envs/azureml-automl/bin:/opt/miniconda/bin:/usr/local/sbi

In [22]:
service.delete()

Running
2024-06-18 16:00:26+00:00 Check and wait for operation (b764fc3b-13dc-4468-b9c5-403f88329e39) to finish.
2024-06-18 16:00:29+00:00 Deleting service entity.
Succeeded


**Submission Checklist**
- I have registered the model.
- I have deployed the model with the best accuracy as a webservice.
- I have tested the webservice by sending a request to the model endpoint.
- I have deleted the webservice and shutdown all the computes that I have used.
- I have taken a screenshot showing the model endpoint as active.
- The project includes a file containing the environment details.
